In [ ]:
## Install libraries
%pip install -Uq upgini catboost

from os.path import exists
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.8/291.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.4/420.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 20.6 MB/s eta 0:00:00

In [ ]:
## Get data
df_path = "train.csv.zip" if exists ("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)


## Data Cleaning and Transformations
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)
df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)

df.head()


,date,store,item,sales
0,2013-01-01,5,24,26
1,2013-01-02,2,7,47
2,2013-01-03,6,21,23
3,2013-01-05,9,37,12
4,2013-01-07,5,49,15


In [ ]:
## Split data into training and testing sets

df = df.sample(n=1000, random_state=0)

# Training dataset - all data from 2013-2016
train = df[df['date'] < "2017-01-01"]

# Testing dataset - all data from 2017
test = df[df['date'] >= "2017-01-01"]

# Take random sample of data
df = df.sample(n=1000, random_state = 0)

#verify both have enough values
print(f"Train rows: {len(train)}, Test rows: {len(test)}")

#split datasets into features and target
train_features = train.drop(columns = ["sales"])
train_target = train["sales"]

test_features = test.drop(columns = ["sales"])
test_target = test["sales"]


Train rows: 798, Test rows: 202


In [ ]:
## Enrich features to account for other relevant variables

from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    cv= CVType.time_series
)

enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
date,All valid,-
target,All valid,-




Running search request, search_id=c7a142f9-7dfe-443f-9577-ce71f28f0b99
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


6 relevant feature(s) found with the search keys: ['date']


f_autofe_roll_2d_min_4199c37ccb,0.5967,100.0000,"120.0, 63.0, 54.0",Training dataset,AutoFE: features from Training dataset,
f_autofe_delta_1d_23f8ab35be,0.1659,39.5210,"70.0, -53.0, -20.0",Training dataset,AutoFE: features from Training dataset,
f_autofe_roll_2d_max_8fce545c58,0.0350,100.0000,"0.0075, 0.0033, 0.0026","Training dataset,Upgini","AutoFE: features from Training dataset,Markets data",Daily
f_financial_date_crude_oil_7d_to_1y_c3e0ad17,0.0084,100.0000,"0.813, 0.7633, 1.1768",Upgini,Markets data,Daily
f_events_date_year_cos1_9014a856,0.0001,100.0000,"-0.6321, -0.06, -0.9442",Upgini,Calendar data,Daily
store,0.0001,100.0000,"10, 9, 1",,,


Training dataset,AutoFE: features from Training dataset,0.7626,2
"Training dataset,Upgini","AutoFE: features from Training dataset,Markets data",0.0350,1
Upgini,Markets data,0.0084,1
Upgini,Calendar data,0.0001,1


Training dataset,f_autofe_roll_2d_min_4199c37ccb,target,roll_2d_min
Training dataset,f_autofe_delta_1d_23f8ab35be,target,delta_1d
"Training dataset,Markets data",f_autofe_roll_2d_max_8fce545c58,f_financial_date_usd_eur_gap_c8eb8d4a,roll_2d_max


In [ ]:
## Define model
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


/usr/local/lib/python3.11/dist-packages/upgini/metrics.py:819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:, self.cat_features] = encoded
/usr/local/lib/python3.11/dist-packages/upgini/metrics.py:819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:, self.cat_features] = encoded
/usr/local/lib/python3.11/dist-packages/upgini/metrics.py:819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:, self.cat_features] = encode

,Dataset type,Rows,Mean target,Baseline MAPE,Enriched MAPE,"Uplift, abs","Uplift, %"
0,Train,501,53.8144,0.570 ± 0.083,0.311 ± 0.053,0.260,45.6%
1,Eval 1,202,61.9109,0.496 ± 0.006,0.291 ± 0.040,0.206,41.4%


In [ ]:
## Train model
enriched_train_features = enricher.transform(train_features, y = train_target, keep_input=True)
enriched_test_features = enricher.transform(test_features, y = test_target, keep_input=True)
enriched_train_features.head()

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 0 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

WARNING #2: 38.8191% of rows (309) in X and eval_set are duplicates with different y values. These rows will be deleted as incorrect
Sample of incorrect row indexes: [0, 1, 7, 8, 10, 11, 12, 13, 15, 16, 17, 18, 22, 24, 28, 34, 37, 38, 41, 49, 50, 57, 58, 64, 69, 74, 75, 76, 77, 78, 79, 85, 89, 95, 97, 98, 101, 102, 103, 104, 105, 111, 114, 118, 120, 122, 124, 128, 129, 130, 131, 132, 140, 143, 144, 146, 147, 150, 151, 153, 157, 159, 163, 164, 165, 167, 168, 169, 170, 171, 174, 181, 182, 187, 188, 190, 192, 200, 201, 202, 203, 205, 207, 208, 209, 210, 211, 213, 215, 219, 220, 221

Column name,Status,Errors
date,All valid,-




Running transform request, id=ec674cce-4a85-4a1b-bbfc-02cbe6efd012
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 0 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

WARNING #2: 38.0000% of rows (76) in X and eval_set are duplicates with different y values. These rows will be deleted as incorrect
Sample of incorrect row indexes: [1, 2, 4, 6, 7, 8, 11, 15, 16, 18, 22, 23, 24, 26, 27, 31, 34, 36, 38, 44, 57, 58, 59, 60, 62, 67, 70, 76, 80, 81, 82, 87, 95, 96, 98, 100, 102, 103, 104, 105, 107, 108, 110, 111, 112, 116, 119, 121, 122, 129, 136, 140, 141, 142, 148, 151, 152, 153, 155, 156, 157, 160, 165, 166, 167, 172, 177, 180, 183, 184, 189, 190, 192, 195, 199, 201]



Column name,Status,Errors
date,All valid,-




Running transform request, id=9e14fad8-d6b4-41b9-90a6-445f8f7e2f04
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,target,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,f_events_date_year_cos1_9014a856,f_autofe_delta_1d_23f8ab35be,f_autofe_roll_2d_max_8fce545c58,f_autofe_roll_2d_min_4199c37ccb
168,2013-11-06,7,24,35,NaN,NaN,NaN,NaN,NaN
532,2015-09-25,4,47,25,NaN,NaN,NaN,NaN,NaN
792,2016-12-22,9,44,28,1.207463,1.000000,NaN,0.0011,28.0
354,2014-11-06,1,25,83,0.816332,0.702527,NaN,-0.0037,83.0
778,2016-12-04,6,42,36,1.157254,0.952635,10.0,0.0000,26.0


In [ ]:
## Model without enrichment
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE") #gives error rate of model

[41.18481124054508]

In [ ]:
## Model with enrichment
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds, "SMAPE") #gives error rate of model

[44.872951181071755]